## SQL - MariaDB

### Packages 

In [1]:
%%bash
echo $VIRTUAL_ENV
pip3 install --upgrade --upgrade-strategy only-if-needed pymysql
pip3 install --upgrade --upgrade-strategy only-if-needed pandas
pip3 install --upgrade --upgrade-strategy only-if-needed sqlalchemy
# pip3 list | wc -l

/home/edu-001/venv/bdl03-1-jpy-3.8
Requirement already up-to-date: pymysql in /home/edu-001/venv/bdl03-1-jpy-3.8/lib/python3.8/site-packages (1.0.2)
Requirement already up-to-date: pandas in /home/edu-001/venv/bdl03-1-jpy-3.8/lib/python3.8/site-packages (1.2.3)
Requirement already up-to-date: sqlalchemy in /home/edu-001/venv/bdl03-1-jpy-3.8/lib/python3.8/site-packages (1.3.23)


### Global configuration

In [2]:
HOST_SQL = 'localhost'
PORT_SQL = 3306
USER_SQL = "myAdmin"
PASS_SQL = "myAdmin"

### pymysql
#### shared code

In [3]:
import pymysql as mariadb

def create_connection_sql(
        host=HOST_SQL,
        port=PORT_SQL,
        database=None,
        user=USER_SQL,
        password=PASS_SQL):
    """ create a database connection to the MariaDB database
        specified by the params
    :param host: the hostname or IP
    :param database: name of the database
    :param user: username
    :param password: password
    :return: connection object or None
    """
    conn = None
    try:
        conn = mariadb.connect(
            host=host, database=database,
            user=user, password=password)
    except mariadb.Error as e:
        print(e)

    return conn

def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row

sql_conn = create_connection_sql(database="Chinook")


#### Just print

In [4]:
sql = """
SELECT Country, COUNT(Country) AS cnt
  FROM Customer
  GROUP BY Country
  HAVING cnt >= 3
  ORDER BY COUNT(Country) DESC
    """

cursor = sql_conn.cursor()
cursor.execute(sql)

for row in iter_row(cursor, 10):
    print(row)

cursor.close()

('USA', 13)
('Canada', 8)
('France', 5)
('Brazil', 5)
('Germany', 4)
('United Kingdom', 3)


#### Pandas

In [5]:
import pandas as pd

sql = """
SELECT Country, COUNT(Country) AS cnt
  FROM Customer
  GROUP BY Country
  HAVING cnt >= 3
  ORDER BY COUNT(Country) DESC
"""

cursor = sql_conn.cursor()
cursor.execute(sql)

# print(cursor.description)

cols = []
for c in cursor.description:
    cols.append(c[0])

cursor.close()

# print(cols)
df = pd.DataFrame(cursor, columns=cols)
df

,Country,cnt
0,USA,13
1,Canada,8
2,France,5
3,Brazil,5
4,Germany,4
5,United Kingdom,3


#### Pandas with SQLalchemy

In [6]:
import pandas as pd
import sqlalchemy

# https://docs.sqlalchemy.org/en/13/dialects/mysql.html#module-sqlalchemy.dialects.mysql.pymysql
# mysql+pymysql://<username>:<password>@<host>/<dbname>[?<options>]
engine = sqlalchemy.create_engine("mysql+pymysql://%s:%s@%s:%s/%s"
                                  % (USER_SQL, PASS_SQL,
                                     HOST_SQL, PORT_SQL,
                                     "Chinook"))

sql = """
SELECT Country, COUNT(Country) AS cnt
  FROM Customer
  GROUP BY Country
  HAVING cnt >= 3
  ORDER BY COUNT(Country) DESC
"""

df = pd.read_sql_query(sql, engine)
df

,Country,cnt
0,USA,13
1,Canada,8
2,Brazil,5
3,France,5
4,Germany,4
5,United Kingdom,3
